# PH 132 Homework 2

Welcome to Homework 2!

The goal of this homework will be to analyze a paper and attempt to replicate the paper's model and findings. We'll be looking at **Prediction of Lifetime Risk for Cardiovascular Disease by Risk Factor Burden at 50 Years of Age**, which you can find [here](https://www.ahajournals.org/doi/full/10.1161/CIRCULATIONAHA.105.548206?url_ver=Z39.88-2003&rfr_id=ori:rid:crossref.org&rfr_dat=cr_pub\%3dpubmed). The assignment will be broken down into two parts: 1) first, we'll be analyzing the paper's approach and methods, then 2) we'll attempt to replicate the findings. You are encouraged to complete all of the questions within the notebook, but if you would like to answer the written questions separately that is fine as long as you **submit all of your responses on Gradescope as 1 PDF and tag the pages appropriately**.

This Homework is due **December 1, 2021 at 11:59pm PT**. Remember you have four slip days over the semester with a maximum of two per homework. If you need an extension for any reason, please reach out (no need to explain any personal details). Submission instructions are the same as for the previous homeworks and details are at the end of the notebook.

Remember, you're encouraged to work with other students in the class, but please list all of the other students you worked with here:

* <*Betsy, Kai, and VivveAnne*>

Please come to office hours and post your questions on Piazza when you need help, remembering to make it private if you are posting code! You can also refer to our classwide [resource sheet](https://piazza.com/class/ksj7rckzlmu1vb?cid=6) for more help.

## Part 1: Written questions

**Question 1. Getting a grasp**

**(a)** What is the objective of the paper? In other words, if the paper is unequivocally successful, what do the authors hope to change about the world?

<font color='red'>The objective of this paper is to estimate lifetime risk factors for CVD to motivate beneficial changes in lifestyle or health behaviors and implement preventative treatments earlier in life. Lifetime risk of CVD will be lowered if we prevent development of risk factors in younger adults.</font>

**(b)** In order to get to that end goal, what variable in the dataset are the authors trying to predict--that is, which variable is the label? Write out the variable name and what it stands for. *Hint: You'll have to look through the [framdoc.pdf](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fpublic-health-196%2Fph196-fa20&urlpath=tree%2Fph196-fa20%2Fhw2%2Fframdoc.pdf&branch=master) documentation.*

<font color='red'>The label to predict is "ANYCHD," which represents any incident of Myocardial infarction (Hospitalized and silent or unrecognized), Angina Pectoris, Cornoary Insufficiency, and fatal Coronary Heart Disease. </font>

**(c)** Does the variable you chose in part b) encompass the entirety of CVD events as defined by the paper? What is it missing or what does it include that it shouldn't?

<font color='red'>ANYCHD does not encompass the entirety of CVD events because it doesn't include congestive heart failure or revascularization procedures without a preceding clinical event. People who underwent these procedures still may have a CVD event later on. Also, ANYCHD includes people with CVD that may have died from a non-CVD event later on. </font>

**(d)** What are some potential biases that could affect the relevance of the label to the paper's true objective?

<font color='red'>Bias could be introduced when we include people who die of non CVD events and don't those who had unrecognized or monitored CVD events.</font>

**(e)** What are some potential biases in the X's of the model? Identify at least 2.

<font color='red'>Selection Bias (as described in the previous question) and reponse bias (lost information beccause patients did return for follow-ups. They included people who take anti-hypertensive medications and those that do not, creating an instance of selection Bias also. </font>

**Question 2. Writing it out**

**(a)** Some patients are on anti-hypertensive medication. We will think of the treatment as $T = {0,1}$. As we learned in lecture, when both treatment statuses are present in our dataset, we have to think about treatment pollution. Is this the case in our dataset? Write out the types of patient observations that are present in our dataset using conditional expectation notation and potential outcomes notation. *Hint: This should be at most two terms.*
    
<font color='red'>Yes, this is the case in our dataset. <br> Factual Y<sub>i</sub> = {Y<sub>0i</sub> if T = 0}    AND   Y<sub>i</sub> = {Y<sub>1i</sub> if T = 1}   <br>Counterfactuals = Y<sub>i</sub> = {Y<sub>0i</sub> if T = 1}  </font>



**(b)** In our prediction task, given a patient's health $X_i$, what we're really trying to predict is the *untreated outcome of patient $i$*. Using conditional expectation notation and potential outcomes notation, write out this term.

<font color='red'> $X_i$ = E [$Y_0i$ | T = 1] - E [$Y_0i$ | T = 0]</font>
    


**(c)** Suppose, instead, that you want to predict which patients would respond well to treatment. Write out the ideal quantity we would like to predict for individual $i$ using potential outcomes notation. *Hint: This should be at most two terms.*

<font color='red'> E [ Y | T=1 ] > E [ Y | T=0 ]</font>

**Question 3. Preparing for analysis**

*Note: All the questions in this section should be answered in at most one sentence.*

**(a)** Before we start doing any analysis on our data, we have to perform data cleaning to make sure it is of the proper type and that we deal with missing values appropriately. What is one reason we have to think carefully about how we handle missing data?
    
<font color='red'>Handling missing data is important because if there was a risk factor that contributed more to lifetime risk factor, but most of the data was missing, we might get a very off prediciton.</font>

**(b)** Before training our model, we have to make sure we have some data to measure the performance of our model. How might we do this?
    
<font color='red'>To measure the performance of our model, we should split the dataset into traning data and testing / validation sets.</font>
    


**(c)** We fit a model on our training data using regularization (e.g. ridge regression). How should we decide on a regularization parameter?
    
<font color='red'>To decide on a regularization parameter, we can perform cross-validation that minimizes the sum of squared residuals to minimize overfitting.</font>

## Part 2: Coding questions

In this section we will use the Framingham dataset to fit a model similar to the one they fit in the paper. For this homework, we're giving you a dataset with only one observation for each patient so that we're only making one prediction per patient.

Let's import necessary libraries and the familiar Framingham dataset:

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
random.seed(132)

In [11]:
fr_raw = pd.read_csv("framingham_small.csv")
fr_raw.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,209.0,52,121.0,70.0,0,0.0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,6238,2,260.0,58,121.0,81.0,0,0.0,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
2,9428,1,283.0,54,141.0,89.0,1,30.0,25.34,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,10552,2,232.0,67,183.0,109.0,1,30.0,30.18,0,...,1,1,2956,2956,2956,2956,2089,2089,2956,0
4,11252,2,343.0,58,155.0,90.0,1,30.0,24.61,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4285


As a reminder, here are all of the columns present in the dataset:

In [12]:
fr_raw.columns

Index(['RANDID', 'SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE',
       'CIGPDAY', 'BMI', 'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'educ',
       'PREVCHD', 'PREVAP', 'PREVMI', 'PREVSTRK', 'PREVHYP', 'TIME', 'PERIOD',
       'HDLC', 'LDLC', 'DEATH', 'ANGINA', 'HOSPMI', 'MI_FCHD', 'ANYCHD',
       'STROKE', 'CVD', 'HYPERTEN', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD',
       'TIMESTRK', 'TIMECVD', 'TIMEDTH', 'TIMEHYP'],
      dtype='object')

## Data Cleaning

We wish to use the following features for our model:

- Systolic Blood Pressure
- Diastolic Blood Pressure
- Total Cholesterol
- Age *(This wasn't used in the paper, but we'll include it since it will be useful later)*
- Diabetic (Yes/No)
- Currently smoking cigarettes (Yes/No)
- Use of Anti-hypertensive medication (Yes/No)


In [13]:
# Drop any rows from fr_raw that have missing values in at least one column.
features = ['SYSBP', 'DIABP', 'TOTCHOL', 'AGE', 'DIABETES', 'CURSMOKE', 'BPMEDS']
fr_clean = fr_raw[features + ['ANYCHD']].dropna()
fr_clean.head()

,SYSBP,DIABP,TOTCHOL,AGE,DIABETES,CURSMOKE,BPMEDS,ANYCHD
0,121.0,70.0,209.0,52,0,0,0.0,1
1,121.0,81.0,260.0,58,0,0,0.0,0
2,141.0,89.0,283.0,54,0,1,0.0,0
3,183.0,109.0,232.0,67,0,1,0.0,0
4,155.0,90.0,343.0,58,0,1,0.0,0


**Question 5.** Next, let's clean up the data types in `fr_clean`. 
- The first four columns (`SYSBP`, `DIABP`, `TOTCHOL`, `AGE`) are imported as floats, but they are actually integer values. 
- The next four columns (`DIABETES`, `CURSMOKE`, `BPMEDS`, `ANYCHD`) are categorical variables; in fact, they are indicator variables (0 if false, 1 for true) and so we can represent this column as integer types as well.


In [14]:
#Convert the column types in `fr_clean` to integers.
fr_clean = fr_clean.astype(int)
fr_clean.head()

,SYSBP,DIABP,TOTCHOL,AGE,DIABETES,CURSMOKE,BPMEDS,ANYCHD
0,121,70,209,52,0,0,0,1
1,121,81,260,58,0,0,0,0
2,141,89,283,54,0,1,0,0
3,183,109,232,67,0,1,0,0
4,155,90,343,58,0,1,0,0


**Question 6.** Finally, let us split the `fr_clean` DataFrame into a train and validation set. 


We would like to use 80% of `fr_clean` for training and the other 20% for validation. 


In [15]:
# Import sklearn functions
from sklearn.model_selection import train_test_split

# Define a function to complete a 80-20 split.
def split_set(df, size):
    
    #df: DataFrame to split
    #size: proportion of data to allocate to validation set (same as train_test_split's test_size)
    
    X = df.drop(columns = 'ANYCHD', axis=1)
    y = df['ANYCHD']
    return train_test_split(X, y, random_state = 132, test_size = size)
    
fr_train, fr_val, y_train, y_val = split_set(fr_clean, size=0.2)
fr_train.shape, fr_val.shape

((2998, 7), (750, 7))

## Model Engineering

For this project, I will use logistic regression, a model that builds on top of linear regression. Instead of predicting in a range of continuous values, logistic regression models "squash" the predictions to be between 0 and 1. We can interpret these predictions as a probability $p$ of being in class 1. In this setting, we can also interpret predictions as a *risk score* for cardiovascular disease. Just like with the OLS regression model, I threshold these predictions to get binary classifications. For example, you might declare a prediction of class 0 if $p < 0.5$ and class 1 if $p \geq 0.5$. 


**Question 7.** Use the `LogisticRegression` class to implement the following:


In [16]:
# Import Logistic Regression class
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define a function to fit the model
def train_logistic_model(X_train, y):
    classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
    model = classifier.fit(X_train, y)
    return model

# Run the fitten model 
model = train_logistic_model(fr_train, y_train)
y_train_pred = model.predict(fr_train)
train_acc = accuracy_score(y_train, y_train_pred)
y_val_pred = model.predict(fr_val)
val_acc = accuracy_score(y_val, y_val_pred)
train_acc, val_acc
print("Training accuracy: {0:.5f} \nValidation accuracy: {1:.5f}".format(train_acc, val_acc))

Training accuracy: 0.72548 
Validation accuracy: 0.73600


## Assigning treatment, Hypothetical

In this section, pretend you're working at a hospital with a limited supply of anti-hypertensive medication. Your boss has noticed that lots of people who receive the medication are still ending up with cardiovascular disease. With only a limited supply, she thinks this medication would be best reallocated to others who would benefit more. She tells you to look at the data and build a model to predict which patients would not benefit from the medication, as measured by whether they go on to have a bad outcome even though they were given the medication. To do this, you’ll fit a model of who goes on to have that bad outcome -- cardiovascular disease — among just the people who were on the medication. Those with high scores will go on to have a bad outcome. Those with the lowest risk scores are the people who would do well on medicaiton, so they are given the medication.

**(Ungraded)** Before you start coding, think about what could go wrong with this approach.

**Question 8.** First, we'll create our training set. From `fr_train` and `y_train`, generate a new DataFrame `fr_train_t1` and corresponding labels `y_train_t1` that contains only patients who are on anti-hypertensive medication.

In [17]:
fr_train_t1 = fr_train[fr_train["BPMEDS"] == 1]
y_train_t1 = y_train[fr_train["BPMEDS"] == 1]
fr_train_t1.shape, y_train_t1.shape

((454, 7), (454,))

**Question 9.** Using the `train_logistic_model` function you wrote in question 4, fit a model on `fr_train_t1` and `y_train_t1`. Assign this new model to a variable `model_t1`. Calculate the accuracy on the training data and on `fr_val` and assign the accuracies to `train_acc_t1` and `val_acc_t1`, respectively.

**(Ungraded)** What do you notice about the accuracies between the treatment group and the validation set (which contains both treated and untreated patients)? Why do you think this is the case?\
The accuracy has more available data in the validation set, which might be why it is more accurate. Also, it is more representative of the whole population.


In [18]:
#fr_train_t1, fr_val_t1, y_train_t1, y_val_t1 = train_test_split(X_t1, y_t1, random_state = 132, test_size = .2)
model_t1 = train_logistic_model(fr_train_t1, y_train_t1)
y_pred_t1 = model_t1.predict(fr_train_t1)
train_acc_t1 = accuracy_score(y_train_t1, y_pred_t1)
# Val accuracy
y_pred_val = model_t1.predict(fr_val)
val_acc_t1 = accuracy_score(y_val, y_pred_val)
print("Training accuracy on treated: {0:.5f} \nValidation accuracy: {1:.5f}".format(train_acc_t1, val_acc_t1))

Training accuracy on treated: 0.61674 
Validation accuracy: 0.71867


**Question 10.** Next, we want to add a column `PROB` to `fr_val` containing the predicted risk scores on the validation set from `model_t1`. You can use the `predict_proba(...)` function then take the column at index 1 (try it out in a separate cell first if you're confused).

*Note:* We're using `predict_proba(...)` instead of `predict(...)` because we want the raw predicted probabilities, not the classifications (0 or 1).

In [19]:
# Predict Probabilities
y_pred_val_proba = model_t1.predict_proba(fr_val)[:,1]
# Extract col at index 1
fr_val['PROB'] = y_pred_val_proba
print(fr_val)

      SYSBP  DIABP  TOTCHOL  AGE  DIABETES  CURSMOKE  BPMEDS      PROB
1498    193    132      280   66         0         0       0  0.450755
2628    140     91      280   67         0         1       0  0.324368
1210    127     76      205   55         1         1       0  0.353548
1791    133     85      276   53         0         1       0  0.253676
1612    135     82      218   61         0         1       1  0.316434
...     ...    ...      ...  ...       ...       ...     ...       ...
1898    205     92      236   64         0         1       1  0.461632
1170    178    108      238   62         0         1       0  0.393074
1703    111     80      261   49         0         1       0  0.211319
1944    147     82      286   58         0         1       0  0.294419
3670    136     88      229   54         0         0       0  0.307966

[750 rows x 8 columns]


**Question 11.** Finally, we can reallocate treatments to those with the lowest risk score, or the highest probability of benefiting from the treatment (or so we think). Create two new DataFrames:

* `old_treat` which contains the rows of `fr_val` corresponding to people who received treatment under the old treatment regime.
* `new_treat` which contains the `len(old_treat)` patients in `fr_val` with the lowest risk scores. *Hint:* the function `sort_values(...)` might be helpful.

*Note:* For simplicity, assume each row is an individual patient even though this is not the case.

In [21]:
old_treat = fr_val[fr_val['BPMEDS'] == 1]
new_treat = fr_val.sort_values('PROB')[:len(old_treat)]
old_treat.shape, new_treat.shape

((118, 8), (118, 8))

**Question 12.** Use the `describe()` function to calculate summary statistics for `old_treat` and `new_treat`. Comment on what you observe. What might be an unintended negative consequence of reallocating treatment in this way?

In [28]:
# use describe() to calculate summary statistics (you will want to use two cells for this)
old_treat.describe()

,SYSBP,DIABP,TOTCHOL,AGE,DIABETES,CURSMOKE,BPMEDS,PROB
count,118.000000,118.000000,118.000000,118.000000,118.000000,118.000000,118.0,118.000000
mean,170.144068,100.805085,277.474576,63.508475,0.169492,0.423729,1.0,0.382014
std,24.860044,12.713314,66.437003,8.527227,0.376785,0.496256,0.0,0.092517
min,119.000000,73.000000,161.000000,47.000000,0.000000,0.000000,1.0,0.174989
25%,150.250000,92.000000,236.000000,57.000000,0.000000,0.000000,1.0,0.318967
50%,168.000000,100.000000,265.500000,63.000000,0.000000,0.000000,1.0,0.378996
75%,186.000000,108.750000,300.000000,69.750000,0.000000,1.000000,1.0,0.429644
max,243.000000,142.000000,558.000000,81.000000,1.000000,1.000000,1.0,0.642736


In [29]:
new_treat.describe()

,SYSBP,DIABP,TOTCHOL,AGE,DIABETES,CURSMOKE,BPMEDS,PROB
count,118.000000,118.000000,118.000000,118.000000,118.000000,118.000000,118.000000,118.000000
mean,138.881356,88.711864,296.237288,56.059322,0.042373,0.711864,0.533898,0.336624
std,22.447724,13.306448,75.379346,8.562782,0.202297,0.454826,0.500977,0.046746
min,95.000000,57.000000,181.000000,40.000000,0.000000,0.000000,0.000000,0.174989
25%,119.000000,80.000000,248.250000,49.000000,0.000000,0.000000,0.000000,0.310552
50%,136.000000,88.000000,282.000000,55.000000,0.000000,1.000000,1.000000,0.355620
75%,156.750000,97.750000,326.000000,62.000000,0.000000,1.000000,1.000000,0.372352
max,197.000000,125.000000,638.000000,76.000000,1.000000,1.000000,1.000000,0.385613


<font color='blue'>An unintended consequence of reallocating treatment in this way is that some patients that would have benefited from medication on the old_treatment scheme, may not end up receiving treatment on the new treatment scheme.</font>

**Question 13.** Use potential outcomes notation to explain why this approach is not ideal. Specifically, fill in the cell below to state what quantity this reallocation procedure uses to assign treatment, and compare this to the ideal quantity that you came up with in Question 2c.

<font color='blue'> This procedure uses a prediction of $Y_{1,i}$ (individual $i$'s risk of CVD when treated) to allocate treatment to those with low predicted risk. Instead, we would like to predict the treatment effect for individual $i$, $Y_{1,i} - Y_{0,i}$, and assign treatment to those with a large negative treatment effect. This is a problem because those who have a low predicted risk score under treatment ($\hat{Y}_{1,i}$) might also have low risk without treatment, meaning they don't actually benefit from treatment that much.</font>